In [2]:
import os
import trimesh
import random

from scripts.models import Segmentation_Network_full
from scripts.utils import save_mesh_with_color_clinical, save_mesh_with_color_acdc, dir_checker
from scripts.dataset import CustomDataset

import torch
import torch.nn as nn


# ACDC

In [4]:
model_dir = './models/clinical_no_TL'
target = 'best' # 'last'
model = torch.load(f'{model_dir}/{target}.pt').cpu()
model.eval()
print()

AttributeError: Can't get attribute 'Segmentation_Network_TL' on <module 'scripts.models' from '/home/vlad/Documents/clean/scripts/models.py'>

In [3]:
model

Segmentation_Network_full(
  (network): Sequential(
    (0): Conv3d(1, 32, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    (1): ReLU()
    (2): Conv3d(32, 64, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    (3): ReLU()
    (4): Conv3d(64, 128, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    (5): ReLU()
    (6): Conv3d(128, 256, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    (7): ReLU()
    (8): Conv3d(256, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1))
    (9): ReLU()
    (10): Conv3d(256, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1))
    (11): ReLU()
    (12): Conv3d(256, 3, kernel_size=(1, 1, 1), stride=(1, 1, 1))
    (13): Sigmoid()
    (14): Flatten(start_dim=1, end_dim=-1)
  )
)

In [2]:
dir_in = './ACDC_simplified/'
path_out = './predictions/ACDC'

test_names = ['patient001', 'patient002', 'patient004', 'patient006', 'patient009', 'patient012', 'patient016', 'patient017', 'patient018', 'patient019', 'patient024', 'patient025', 'patient026', 'patient027', 'patient028', 'patient033', 'patient035', 'patient036', 'patient037', 'patient040', 'patient041', 'patient043', 'patient045', 'patient046', 'patient048', 'patient051', 'patient053', 'patient055', 'patient057', 'patient059', 'patient061', 'patient065', 'patient067', 'patient069', 'patient070', 'patient071', 'patient072', 'patient073', 'patient078', 'patient079', 'patient081', 'patient082', 'patient083', 'patient085', 'patient086', 'patient090', 'patient091', 'patient093', 'patient097', 'patient100']
name = random.choice(test_names) + '.obj'
preprocessed_dir = 'prepared_data/sdf'
preprocessed_dir += '/' + name[:-4]

mesh = trimesh.load(dir_in+name)
print('Infering on', name)

Infering on patient035.obj


In [ ]:
# dir_in = './ACDC_simplified/'
# path_out = './predictions/ACDC'

# test_names = ['patient001', 'patient002', 'patient004', 'patient006', 'patient009', 'patient012', 'patient016', 'patient017', 'patient018', 'patient019', 'patient024', 'patient025', 'patient026', 'patient027', 'patient028', 'patient033', 'patient035', 'patient036', 'patient037', 'patient040', 'patient041', 'patient043', 'patient045', 'patient046', 'patient048', 'patient051', 'patient053', 'patient055', 'patient057', 'patient059', 'patient061', 'patient065', 'patient067', 'patient069', 'patient070', 'patient071', 'patient072', 'patient073', 'patient078', 'patient079', 'patient081', 'patient082', 'patient083', 'patient085', 'patient086', 'patient090', 'patient091', 'patient093', 'patient097', 'patient100']

# for name in test_names:
#     name = name + '.obj'
    
#     preprocessed_dir = 'prepared_data/sdf'
#     preprocessed_dir += '/' + name[:-4]
    
#     mesh = trimesh.load(dir_in+name)
#     print('Infering on', name)

    
#     if os.path.exists(preprocessed_dir):
#         sdf = torch.load(f'{preprocessed_dir}/sdf.pt')
#         colors = torch.load(f'{preprocessed_dir}/color.pt')
#     else:
#         datagen = CustomDataset(name = name,
#                       device='cpu',
#                       count=20,
#                       rotate_angle=1,
#                       dataset = 'acdc',
#                       num_classes=3,
#                       inference = True,
#                       return_points = True)
        
#         sdf, colors, points = datagen.get_data()
#         os.mkdir(preprocessed_dir)
#         torch.save(sdf, f'{preprocessed_dir}/sdf.pt')
#         torch.save(colors, f'{preprocessed_dir}/color.pt')
    

In [4]:
if os.path.exists(preprocessed_dir):
    sdf = torch.load(f'{preprocessed_dir}/sdf.pt')
    colors = torch.load(f'{preprocessed_dir}/color.pt')
else:
    datagen = CustomDataset(name = name,
                  device='cpu',
                  count=20,
                  rotate_angle=1,
                  dataset = 'acdc',
                  num_classes=3,
                  inference = True,
                  return_points = True)
    
    sdf, colors, points = datagen.get_data()
    os.mkdir(preprocessed_dir)
    torch.save(sdf, f'{preprocessed_dir}/sdf.pt')
    torch.save(colors, f'{preprocessed_dir}/color.pt')
    

In [5]:
predicted = model(sdf.float())

In [6]:
colors = torch.argmax(colors, dim=1)
predicted = torch.argmax(predicted, dim=1)

In [7]:
dirname = dir_checker(path_out, name)
save_mesh_with_color_acdc(mesh, colors, path=f'{dirname}/{name[:-4]}_original.obj')
save_mesh_with_color_acdc(mesh, predicted, path=f'{dirname}/{name[:-4]}_predicted.obj')

There's already a dirrectory with this name!
Folder for predictions: ./predictions/ACDC/patient035 (1)


# Clinical

In [3]:
model_dir = './models/clinical_no_TL'
path_out = './predictions/Clinical'
target = 'best' # 'last'
model = torch.load(f'{model_dir}/{target}.pt').cpu()
model.eval()
print()

In [3]:
dir_in = './data_for_transfer_learning/obj/'
test_names = ['003', '009', '011', '012', '014', '016', '017', '021', '023', '026', '029', '030', '031', '032', '033', '034', '035', '037']

#name = random.choice(test_names) + '.obj'
name = '030.obj'

preprocessed_dir = 'TL_prepared_data/sdf'
preprocessed_dir += '/' + name[:-4]

mesh = trimesh.load(dir_in+name)
print('Infering on', name)

Infering on 030.obj


In [4]:
from tqdm.notebook import tqdm

In [7]:
dir_in = './data_for_transfer_learning/obj/'
test_names = ['003', '009', '011', '012', '014', '016', '017', '021', '023', '026', '029', '030', '031', '032', '033', '034', '035', '037']


preprocessed_dir = 'TL_prepared_data/sdf'


for name in tqdm(test_names):
    name = name + '.obj'
    
    preprocessed_dir = 'TL_prepared_data/sdf'
    preprocessed_dir += '/' + name[:-4]
    
    mesh = trimesh.load(dir_in+name)
    print('Infering on', name)

    
    if os.path.exists(preprocessed_dir):
        sdf = torch.load(f'{preprocessed_dir}/sdf.pt')
        colors = torch.load(f'{preprocessed_dir}/color.pt')
    else:
        datagen = CustomDataset(name = name,
                          device='cpu',
                          count=20,
                          rotate_angle=1,
                          dataset = 'clinical',
                          num_classes=4,
                          inference = True,
                          return_points = True,
                          data_dir = './data_for_transfer_learning/obj')
        
        sdf, colors, points = datagen.get_data()
        os.mkdir(preprocessed_dir)
        torch.save(sdf, f'{preprocessed_dir}/sdf.pt')
        torch.save(colors, f'{preprocessed_dir}/color.pt')
    

  0%|          | 0/18 [00:00<?, ?it/s]

Infering on 003.obj
Infering on 009.obj
Infering on 011.obj
Infering on 012.obj
Infering on 014.obj
Infering on 016.obj
Infering on 017.obj
Infering on 021.obj
Infering on 023.obj
Infering on 026.obj
Infering on 029.obj
Infering on 030.obj
Infering on 031.obj
Infering on 032.obj
Infering on 033.obj
Infering on 034.obj
Infering on 035.obj
Infering on 037.obj


In [10]:
if os.path.exists(preprocessed_dir):
    sdf = torch.load(f'{preprocessed_dir}/sdf.pt')
    colors = torch.load(f'{preprocessed_dir}/color.pt')
else:
    datagen = CustomDataset(name = name,
                      device='cpu',
                      count=20,
                      rotate_angle=1,
                      dataset = 'clinical',
                      num_classes=4,
                      inference = True,
                      return_points = True,
                      data_dir = './data_for_transfer_learning/obj')
    
    sdf, colors, points = datagen.get_data()
    os.mkdir(preprocessed_dir)
    torch.save(sdf, f'{preprocessed_dir}/sdf.pt')
    torch.save(colors, f'{preprocessed_dir}/color.pt')
    

In [11]:
predicted = model(sdf.float())

In [12]:
colors = torch.argmax(colors, dim=1)
predicted = torch.argmax(predicted, dim=1)

In [13]:
dirname = dir_checker(path_out, name)
save_mesh_with_color_clinical(mesh, colors, path=f'{dirname}/{name[:-4]}_original.obj')
save_mesh_with_color_clinical(mesh, predicted, path=f'{dirname}/{name[:-4]}_predicted.obj')

There's already a dirrectory with this name!
Folder for predictions: ./predictions/Clinical/030 (1)


# Clinical With TL

In [14]:
model_dir = './models/clinical_TL'
path_out = './predictions/Clinical_TL'
target = 'best' # 'last'
model = torch.load(f'{model_dir}/{target}.pt').cpu()
model.eval()
print()

In [15]:
predicted = model(sdf.float())

In [16]:
predicted = torch.argmax(predicted, dim=1)

In [17]:
dirname = dir_checker(path_out, name)
save_mesh_with_color_clinical(mesh, colors, path=f'{dirname}/{name[:-4]}_original.obj')
save_mesh_with_color_clinical(mesh, predicted, path=f'{dirname}/{name[:-4]}_predicted.obj')

There's already a dirrectory with this name!
Folder for predictions: ./predictions/Clinical_TL/030 (0)
